In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

In [ ]:
# Cargar el conjunto de datos
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url)

In [ ]:
# Pasamos los datos a formato fecha
df['Month'] = pd.to_datetime(df['Month'], format = '%Y-%m')

In [ ]:
# Indicamos el mes como índice de la tabla
df = df.set_index('Month')

In [ ]:
df = df.squeeze()

In [ ]:
df.head()

In [ ]:
# Visualizar el conjunto de datos
plt.figure(figsize=(24, 6))
plt.plot(df)
plt.title('Pasajeros Aéreos Mensuales (1949-1960)')
plt.xlabel('Fecha')
plt.ylabel('Número de Pasajeros')
plt.grid(True)
plt.show()
plt.close()

In [ ]:
# Valor real
def real_value(year, month):
    date = f'{year}-{month}'
    row = df.loc[date]
    return row.sum()

In [ ]:
# Dato inmediatamente anterior
def last_record(year, month):
    date = f'{year}-{month}'
    last_record_row = df.loc[df.index < date].tail(1)
    return last_record_row.sum()

In [ ]:
# Promedio de todos los datos anteriores
def mean_from_past_data(year, month):
    date = f'{year}-{month}'
    past_data = df.loc[df.index < date]
    mean = past_data.mean()
    return round(mean)

In [ ]:
# Promedio de los último n datos
def last_n_records(year, month, n_periods):
    date = f'{year}-{month}'
    last_n_records_rows = df.loc[df.index < date].tail(n_periods)
    return round(last_n_records_rows.mean())

In [ ]:
# Ponderación de los datos pasados

## Definimos una función que suaviza una serie de tiempo dado un alpha
def exponential_smoothing(series, alpha):
    smoothed_series = pd.Series(index = series.index, dtype = 'float64')
    smoothed_series[0] = series[0]

    for t in range(1, len(series)):
        smoothed_series[t] = alpha * series[t] + (1 - alpha) * smoothed_series[t - 1]

    return smoothed_series

## Definimos una función que suaviza una serie de tiempo dado un alpha y luego
## genera una predicción para los siguientes n periodos
def exponential_smoothing_forecast(series, alpha, steps):
    smoothed_series = exponential_smoothing(series, alpha)
    last_smoothed_value = smoothed_series[-1]

    forecast_values = []
    for _ in range(steps):
        forecast_values.append(last_smoothed_value)
        last_smoothed_value = alpha * series[-1] + (1 - alpha) * last_smoothed_value

    return forecast_values

def ses_forecast(year, month, alpha = 0.6):
    date = f'{year}-{month}'
    past_data = df.loc[df.index < date]
    return round(exponential_smoothing_forecast(past_data, alpha, 1)[0])

In [ ]:
# Declara una función que, dado un año y un mes, pronostica el número de pasajeros
# (Las fechas no pueden ser superiores a Diciembre de 1961)
def print_forecasts(year, month, n_periods = 6, alpha = 0.6):
    print(f'Pronóstico real: {real_value(year, month)}')
    print(f'Pronóstico por dato anterior: {last_record(year, month)}')
    print(f'Pronóstico por promedio histórico: {mean_from_past_data(year, month)}')
    print(f'Pronóstico por promedio movil: {last_n_records(year, month, n_periods)}')
    print(f'Pronóstico por promedio ponderado: {ses_forecast(year, month, alpha)}')

print_forecasts(1950, 12)

## Respuesta ChatGPT

### Pregunta

Genera una fucnión en python que, dada una colección de datos en una serie de tiempo, pronostique el valor para el siguiente periodo. Esto usando la metodología de suavizamiento exponencial

### Respuesta

Por supuesto, puedo ayudarte a crear una función en Python que utilice el método de suavizamiento exponencial para predecir el valor para el siguiente período en una serie de tiempo. Aquí tienes un ejemplo de cómo podrías implementar esto utilizando la librería pandas y los cálculos de suavizamiento exponencial:

```python
import pandas as pd

def exponential_smoothing(series, alpha):
    """
    Realiza suavizamiento exponencial en una serie de tiempo.
    
    :param series: La serie de tiempo a suavizar.
    :param alpha: El factor de suavización (0 <= alpha <= 1).
    :return: La serie de tiempo suavizada.
    """
    smoothed_series = pd.Series(index=series.index)
    smoothed_series[0] = series[0]  # El primer valor se mantiene igual
    
    for t in range(1, len(series)):
        smoothed_series[t] = alpha * series[t] + (1 - alpha) * smoothed_series[t - 1]
    
    return smoothed_series

def exponential_smoothing_forecast(series, alpha, steps):
    """
    Realiza una predicción utilizando suavizamiento exponencial en una serie de tiempo.
    
    :param series: La serie de tiempo para la predicción.
    :param alpha: El factor de suavización (0 <= alpha <= 1).
    :param steps: El número de períodos a pronosticar.
    :return: Una lista de valores pronosticados.
    """
    smoothed_series = exponential_smoothing(series, alpha)
    last_smoothed_value = smoothed_series[-1]
    
    forecast_values = []
    for _ in range(steps):
        forecast_values.append(last_smoothed_value)
        last_smoothed_value = alpha * series[-1] + (1 - alpha) * last_smoothed_value
    
    return forecast_values

# Ejemplo de uso
data = [10, 15, 20, 25, 30, 35]
date_index = pd.date_range(start='2023-08-01', periods=len(data))
series = pd.Series(data, index=date_index)

alpha = 0.2  # Factor de suavización
steps = 1   # Número de períodos a pronosticar

forecast = exponential_smoothing_forecast(series, alpha, steps)
print("Serie de tiempo original:")
print(series)
print("\nValor pronosticado para el siguiente período:")
print(forecast)
```

En este ejemplo, `exponential_smoothing()` calcula la serie suavizada utilizando el método de suavizamiento exponencial. Luego, `exponential_smoothing_forecast()` utiliza la serie suavizada para predecir el valor para el siguiente período. Puedes ajustar el valor de `alpha` para controlar el peso de los datos pasados en la predicción.